In [1]:
from __future__ import print_function
from __future__ import division

import os
import math

import numpy as np
from astropy.io import fits
from astropy.table import Table
from scipy.stats import binned_statistic, scoreatpercentile
import pickle
from scipy.optimize import minimize
import time
from voronoi_2d_binning import voronoi_2d_binning
from sklearn.neighbors import NearestNeighbors
import pickle
import imp # reload modules if necessary

In [2]:
import binning
import bin_debiasing
import fit_debiasing
import make_dictionaries
import params

In [3]:
% matplotlib inline
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['figure.figsize'] = (10.0, 8)
mpl.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

In [4]:
os.mkdir('output_files/') if os.path.isdir('output_files/') is False else None

source_directory = params.source_directory
full_sample = params.full_sample

min_log_fv = -1.5
max_log_fv = 0.01 


In [5]:
full_data = Table.read(source_directory + full_sample)
print('Loaded galaxy data...')



Loaded galaxy data...


In [6]:
#only use data which have z,Mr, and r50 data
z_range = (full_data['REDSHIFT_1']>0)
m_range = (full_data['PETROR50_R_KPC']>0)
r_range = (full_data['PETROMAG_MR']>-25)

full_data = full_data[z_range & m_range & r_range]

In [7]:
questions = make_dictionaries.questions
print('Loaded questions...')
function_dictionary = make_dictionaries.function_dictionary
print('Loaded functions...')

Loaded questions...
Loaded functions...


In [8]:
questions

{'t00_smooth_or_features': {'answerlabels': ['Smooth', 'Features', 'Artifact'],
  'answers': ['a0_smooth', 'a1_features', 'a2_artifact'],
  'pre_answers': None,
  'pre_questions': None,
  'questionlabel': 'Smooth or features'},
 't01_disk_edge_on': {'answerlabels': ['Yes', 'No'],
  'answers': ['a0_yes', 'a1_no'],
  'pre_answers': ['a1_features'],
  'pre_questions': ['t00_smooth_or_features'],
  'questionlabel': 'Edge on'},
 't02_bar': {'answerlabels': ['Yes', 'No'],
  'answers': ['a0_bar', 'a1_no_bar'],
  'pre_answers': ['a1_features', 'a1_no'],
  'pre_questions': ['t00_smooth_or_features', 't01_disk_edge_on'],
  'questionlabel': 'Bar'},
 't03_spiral': {'answerlabels': ['Yes', 'No'],
  'answers': ['a0_spiral', 'a1_no_spiral'],
  'pre_answers': ['a1_features', 'a1_no'],
  'pre_questions': ['t00_smooth_or_features', 't01_disk_edge_on'],
  'questionlabel': 'Spiral'},
 't04_bulge_prominence': {'answerlabels': ['None',
   'Noticeable',
   'Obvious',
   'Dominant'],
  'answers': ['a0_no_bulg

In [15]:
def reduce_sample(full_data,questions,question,p_cut=0.3,N_cut=5,normalised_values=True):
    ''' Reduce the sample to p>p_cut galaxies only, 
        using the precedeing questions'''
    
    previous_q = questions[question]['pre_questions']
    previous_a = questions[question]['pre_answers']
    
    if normalised_values == True:
        suffix = '_debiased_rh_normalised'
    else:
        suffix = '_debiased_rh'
    
    if previous_q != None:
        
        p_col = np.ones(len(full_data))
        
        for m in range(len(previous_q)):
            p_col = p_col*(full_data[previous_q[m] + '_' + previous_a[m] + suffix])
        N_col = (full_data[question + '_count_weighted']) #change: instead of 5 people voting for the answer that leads to this question, just require 5 vote on this question
        
        select = (p_col > p_cut) & (N_col >= N_cut)
        data_reduced = full_data[select]
        print('{}/{} galaxies with p>{} and N>={}.'.format(len(data_reduced),
                                                          len(full_data),p_cut,N_cut))
    
    else:
        data_reduced = full_data.copy()
        print('Primary question, so all {} galaxies used.'.format(len(data_reduced)))
    
    return data_reduced

In [10]:
def get_bins(question,answer):
    ''' Get bins if they have already been created from a 
        previous running of the debiasing'''
    
    bins = Table.read('output_files/'+ question + '/' + answer + '/bins.fits')
    all_bins = Table.read('output_files/'+ question + '/' + answer + '/all_bins.fits')
    vbins_table = Table.read('output_files/'+ question + '/' + answer + '/vbin_parameters.fits')
    
    vbins = bins['vbin']
    zbins = bins['zbin']
    zbins_coarse = bins['coarse_zbin']
    vbins_all = all_bins['vbin']
    zbins_all = all_bins['zbin']
    zbins_coarse_all = all_bins['coarse_zbin']
    
    return vbins,zbins,zbins_coarse,vbins_all,zbins_all,zbins_coarse_all,vbins_table

In [11]:
def histogram_fractions(data,hist_bins):
    ''' Get raw histogram values '''
    h,bin_edges = np.histogram(data,bins=hist_bins)
    f = h/np.sum(h)
    return f


def bin_by_column(column, nbins, fixedcount=True):
    ''' Bin the data into redshift slices 
    (or by any column) '''
    
    sorted_indices = np.argsort(column)
    if fixedcount:
        bin_edges = np.linspace(0, 1, nbins + 1)
        bin_edges[-1] += 1
        values = np.empty(len(column))
        values[sorted_indices] = np.linspace(0, 1, len(column))
        bins = np.digitize(values, bins=bin_edges)
    else:
        bin_edges = np.linspace(np.min(column),np.max(column), nbins + 1)
        bin_edges[-1] += 1
        values = column
        bins = np.digitize(values, bins=bin_edges)
    x, b, n = binned_statistic(values, column, bins=bin_edges)
    return x, bins


def get_rms(dataset,reference,redshifts):
    ''' Calculate rms of a dataset in comparison w. reference data'''
    hist_bins = np.linspace(0,1,11)
    hist_bins[0] = -1
    hist_bins[-1] = 2
    zv,zb = bin_by_column(redshifts,nbins=10)
    
    rms_values = np.zeros(len(np.unique(zb)))
    ref_fractions = histogram_fractions(reference,hist_bins)
    
    for i,z in enumerate(np.unique(zb)):
        
        sample = dataset[zb == z]
        fractions = histogram_fractions(sample,hist_bins)
        rms_values[i] = np.sum((fractions-ref_fractions)**2)
    
    return np.sum(rms_values)

In [12]:
def choose_best_function(raw_data,debiased,question,answer):
    ''' Decide on which set of debiasing values is preferred'''
    volume_ok = raw_data['in_volume_limit'] == 1
    
    # Raw data for reference:
    vl  = raw_data[volume_ok][question + '_' + answer + '_weighted_fraction']
    
    # 2 sets of debiased data for comparison:
    vl_bin = debiased['bin_method'][volume_ok]
    vl_fit = debiased['fit_method'][volume_ok]
    redshifts = full_data['REDSHIFT_1'][volume_ok]
    
    low_z = (redshifts >= 0.03) & (redshifts < 0.035)
    reference = vl[low_z] # raw low-z for comparison
    
    rms_bin = get_rms(vl_bin,reference,redshifts) # get rms values
    rms_fit = get_rms(vl_fit,reference,redshifts) # get rms values
    
    print('rms^2(bin) = {0:.3f}'.format(rms_bin))
    print('rms^2(fit) = {0:.3f}'.format(rms_fit))
    if rms_bin < rms_fit:
        print('---> bin method selected')
        debiased_values = debiased['bin_method']
    else:
        print('---> fit method selected')
        debiased_values = debiased['fit_method'] # Prefer fit method if it 
        # has better rms^2 value.
        
    return debiased_values

In [13]:
def bin_and_debias(full_data,question,questions,answer
                   ,bins_exist=False,n_per_bin=100,coarse=False):
    '''Set to 'coarse' to make the fitting only apply to the 'coarse binning'of 4 redshift bins per 
    voronoi bin rather than the fully binned data'''
    
    # Create output file folders:
    (os.mkdir('output_files/'+ question) if
     os.path.isdir('output_files/'+ question) is False else None)
    (os.mkdir('output_files/'+ question + '/' + answer) if
     os.path.isdir('output_files/'+ question + '/' + answer) is False else None)
    
    # Only use the p>0.5 and N>= 5 data for a given question:
    data = reduce_sample(full_data,questions,question)
    
    if bins_exist == True:
        (vbins,zbins,zbins_coarse,
         vbins_all,zbins_all,zbins_coarse_all,
         vbins_table) = get_bins(question,answer)
        print('Bins obtained from previous iteration...')
        
    else:
        (vbins,zbins,zbins_coarse,
         vbins_all,zbins_all,zbins_coarse_all,
         vbins_table) = binning.bin_data(data,full_data,
                                        question,answer,
                                        signal=n_per_bin)
        
    # Save the binning data:
    bin_table = Table([vbins,zbins,zbins_coarse],names=('vbin','zbin','coarse_zbin'))
    all_bin_table = Table([vbins_all,zbins_all,zbins_coarse_all],names=('vbin','zbin','coarse_zbin'))
    bin_table.write('output_files/'+ question + '/' + answer + '/bins.fits',overwrite=True)
    all_bin_table.write('output_files/'+ question + '/' + answer + '/all_bins.fits',overwrite=True)
    vbins_table.write('output_files/'+ question + '/' + answer + '/vbin_parameters.fits',overwrite=True)

    # Debias using the discrete bin method 1st:
    debiased_bin = bin_debiasing.debias(data,full_data,vbins,zbins,vbins_all,zbins_all,question,answer)
    # Now debias using the functional fitting method:
    debiased_fit,dout,fit_setup,zbins,fit_vbin_results = fit_debiasing.debias_by_fit(data,full_data,vbins,zbins,
                                                                                     zbins_coarse,question,answer,
                                                                                     function_dictionary,min_log_fv,
                                                                                     coarse=coarse)
    
    volume_ok = data['in_volume_limit'] == 1    
    vl_data = full_data[volume_ok]
    vl_fit = debiased_fit[volume_ok]
    vl_bin = debiased_bin[volume_ok]

    debiased_table = Table([debiased_bin,debiased_fit],names=('bin_method','fit_method'))
    debiased_table.write('output_files/'+ question + '/' + answer + '/debiased.fits',overwrite=True)
    dout.write('output_files/'+ question + '/' + answer + '/fit_results.fits',overwrite=True)
    pickle.dump(fit_setup,open('output_files/'+ question + '/' + answer + '/fit_setup.p', "wb" ))
    
    return debiased_table

In [16]:
question_order = ['t10_arms_number']

'''question_order = ['t00_smooth_or_features'
                  ,'t01_disk_edge_on'
                  ,'t07_rounded'
                  ,'t02_bar'
                  ,'t03_spiral'
                  ,'t08_bulge_shape'
                  ,'t04_bulge_prominence'
                  ,'t09_arms_winding'
                  ,'t10_arms_number'
                  ,'t05_odd'
                  ,'t06_odd_feature']'''

for question in question_order:
    answers = questions[question]['answers']
    
    for answer in answers:
        
        #bins_exist = os.path.isfile('output_files/'+ question + '/' + answer + '/bins.fits')
        bins_exist = False
        
        print('----------------------------------')
        print('Question to be debiased:',question)
        print('Answer to be debiased:',answer)
        
        debiased = bin_and_debias(full_data,question,questions,answer,
                                  bins_exist=bins_exist,n_per_bin=50,coarse=False) # Use smaller bins!
        
        deb_vals = choose_best_function(full_data,debiased,question,answer)
        full_data[question + '_' + answer + '_debiased_rh'] = deb_vals
        
        print('----------------------------------')

    debiased_values = np.array([full_data[question + '_' + a + '_debiased_rh'] for a in answers])
    debiased_norm = debiased_values/np.sum(debiased_values,axis=0)
    debiased_norm[np.isnan(debiased_norm)] = 0
    for m in range(len(debiased_norm)):
        full_data[question + '_' + answers[m] + '_debiased_rh_normalised'] = debiased_norm[m]


----------------------------------
Question to be debiased: t10_arms_number
Answer to be debiased: a0_1
6480/60783 galaxies with p>0.3 and N>=5.
Bin-accretion...
60  initial bins.
Reassign bad bins...
28  good bins.
Modified Lloyd algorithm...
8

binning.py:33: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  R50_bin_coords = R50_bin_centres.repeat(n_rect_bins).reshape(n_rect_bins, n_rect_bins).ravel()
binning.py:33: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  R50_bin_coords = R50_bin_centres.repeat(n_rect_bins).reshape(n_rect_bins, n_rect_bins).ravel()
binning.py:34: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Mr_bin_coords = Mr_bin_centres.repeat(n_rect_bins).reshape(n_rect_bins, n_rect_bins).T.ravel()
binning.py:34: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Mr_bin_coords = Mr_bin_centres.repeat(n_rect_bins).reshape(n_rect_bins, n_rect_bins).T.ravel()
binning.py:74: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape

  iterations.
Unbinned pixels:  0  /  538
Fractional S/N scatter (%): 13.3513822837
28 voronoi bins
1.75 redshift bins per voronoi bin
Using fixed width bins
All bins fitted! 14.7529640198s in total
chisq(logistic) = 0.00111694015384
All bins fitted! 11.1984770298s in total
chisq(exp. power) = 0.000795864747855
All bins fitted! 13.0506298542s in total
Selected functions:------

bin_debiasing.py:76: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 60783 but corresponding boolean dimension is 6480
  debiased_column[data_column == 0] = 0 # Don't 'debias up' 0s.
bin_debiasing.py:77: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 60783 but corresponding boolean dimension is 6480
  debiased_column[data_column == 1] = 1 # Don't 'debias down' the 1s.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:45: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 60783 but corresponding boolean dimension is 6480



k: linear(M),log(R),log(z)
c: linear(M),linear(R),linear(z)
rms^2(bin) = 0.063
rms^2(fit) = 0.010
---> fit method selected
----------------------------------
----------------------------------
Question to be debiased: t10_arms_number
Answer to be debiased: a1_2
6480/60783 galaxies with p>0.3 and N>=5.

...

binning.py:33: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  R50_bin_coords = R50_bin_centres.repeat(n_rect_bins).reshape(n_rect_bins, n_rect_bins).ravel()
binning.py:34: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Mr_bin_coords = Mr_bin_centres.repeat(n_rect_bins).reshape(n_rect_bins, n_rect_bins).T.ravel()
binning.py:34: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Mr_bin_coords = Mr_bin_centres.repeat(n_rect_bins).reshape(n_rect_bins, n_rect_bins).T.ravel()
binning.py:74: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.
  rect_bin_count[ok_bin] = count
binning.py:33: DeprecationWarning: using a non-integer number instead of an integ


Bin-accretion...
93

binning.py:33: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  R50_bin_coords = R50_bin_centres.repeat(n_rect_bins).reshape(n_rect_bins, n_rect_bins).ravel()
binning.py:34: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Mr_bin_coords = Mr_bin_centres.repeat(n_rect_bins).reshape(n_rect_bins, n_rect_bins).T.ravel()
binning.py:34: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Mr_bin_coords = Mr_bin_centres.repeat(n_rect_bins).reshape(n_rect_bins, n_rect_bins).T.ravel()
binning.py:74: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.
  rect_bin_count[ok_bin] = count


  initial bins.
Reassign bad bins...
26  good bins.
Modified Lloyd algorithm...
21  iterations.
Unbinned pixels:  0  /  1066
Fractional S/N scatter (%): 14.6377757481
26 voronoi bins
3.84615384615 redshift bins per voronoi bin
All bins fitted! 11.4079971313s in total
chisq(logistic) = 0.0170828725295
All bins fitted! 15.1893730164s in total
chisq(exp. power) = 0.00228401426628
All bins fitted! 9.55543112755s in total
Selected functions:------
k: exp(M),exp(R),log(z)
c: linear(M),log(R),log(z)
rms^2(bin) = 1.367
rms^2(fit) = 0.059
---> fit method selected
----------------------------------


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:36: RuntimeWarning: invalid value encountered in true_divide


In [17]:
outname = 'output_files/debiase_test_2_'+question+'.fits'
outname

'output_files/debiase_test_2_t10_arms_number.fits'

In [18]:
full_data.write(outname)